In [61]:
import yahooquery
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

CRITICAL TODO: 
shuffle windows

the problem with most LSTM based stock price prediction is:

In [45]:
# get previous quarters open and close prices
# never use same data twice
# always use data from before more than 1 month ago
# 3 months for a sample, predict 1 month. This gives us 4 samples per ticker per year.
# easiest additional feature to add right now is volume. 200 day MA or 200 week MA could be interesting features later.
# features must be unique to each sample (using macro data like inflation would cause model to overfit to one specific instance in history, since the data will be the same for each ticker)
# yahooquery priceHistory is sorted by date already, but add tests for this later

other ways to transform data:
%change day to day.

In [46]:
totalDays = 40
trainDays = 30
testDays = totalDays - trainDays

In [47]:
tickers = ['SPY', 'AAPL', 'SNAP', 'TSLA']
dfs = []
for ticker in tickers:
    priceHistory = yahooquery.Ticker(ticker, asnychronous=True)
    df = priceHistory.history(period='max', interval='1d')
    df = df.dropna()
    for col in ['date','high', 'low','adjclose','dividends', 'splits']:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)
    dfs.append(df)

    # TODO: if splits, separate into 2 different dfs. no samples should overlap a split.

print(len(dfs[0]))


7690


In [57]:
# normalize data 0-1
sc_price = MinMaxScaler()
sc_volume = MinMaxScaler()
for df in dfs:
    df[['open', 'close']] = sc_price.fit_transform(df[['open','close']])
    df[['volume']] = sc_price.fit_transform(df[['volume']])
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# use inverse_transform to interpret predictions later


# separate priceHistories into 40 days chunks

In [58]:
# combine all tickers into 40 day chunks
windows = []
for df in dfs:
    for i in range(df['close'].count() // totalDays): # 30 market days training, 10 market days testing
        windows.append(df.iloc[i*totalDays : (i+1)*totalDays])

print(len(windows))

582


In [59]:

windowsValues = [df.values for df in windows] 
# float 32 is default ^, can reduce open and close to 16 if not normalizing later
print(len(windowsValues))
print(len(windowsValues[0]))


# # VSTACK puts all data back into shape (N*30, 3) for X and (N*10, 3) for Y. not what we want.
# X = np.vstack([arr[ : trainDays] for arr in windowsValues]) # vstack concatenates along first axis, turning (N,) into (1,N)
# Y = np.vstack([arr[trainDays : ] for arr in windowsValues])
# print(X.shape)
# print(Y.shape)


# split 30 into X and 10 into Y for each window
X = np.array([arr[ : trainDays] for arr in windowsValues]) # vstack concatenates along first axis, turning (N,) into (1,N)
Y = np.array([arr[trainDays : ] for arr in windowsValues])
print(X.shape)
print(Y.shape)

582
40
(582, 30, 3)
(582, 10, 3)


In [60]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

trainX_tensor = torch.Tensor(X_train)
trainY_tensor = torch.Tensor(Y_train)
testX_tensor = torch.Tensor(X_test)
testY_tensor = torch.Tensor(Y_test)



In [69]:
train_data = TensorDataset(trainX_tensor, trainY_tensor)
test_data = TensorDataset(testX_tensor, testY_tensor)

train_loader = DataLoader(train_data, batch_size=5, shuffle=True) # shuffle since samples are taken sequentially but should be evaluated without patterns in the sequence.
test_loader = DataLoader(test_data, batch_size=5, shuffle=False) # doesn't change model so don't need to shuffle.